## 1. 프로젝트 생성

In [1]:
# !pip install fake_useragent scrapy-fake-useragent

In [2]:
import scrapy, requests, re
import pandas as pd
from scrapy.http import TextResponse
from selenium import webdriver

/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
!rm -rf musinsa
!scrapy startproject musinsa

New Scrapy project 'musinsa', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/notebooks/project/crawling-repo-3/musinsa

You can start your first spider with:
    cd musinsa
    scrapy genspider example example.com


In [4]:
!ls

category_crawler.ipynb	mss-crawler-top-selenium.ipynb	musinsa_2
datas			mss_category_crawler.py		sizeinfo
item_crawler.ipynb	musinsa


In [5]:
!tree musinsa/

musinsa/
├── musinsa
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


## 2. xpath 확인

### link

In [6]:
req = requests.get("https://search.musinsa.com/category/001003")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [7]:
links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
len(links), links[0]

(90, 'https://store.musinsa.com/app/goods/855195')

In [8]:
# 페이지 당 개수는 최대 90개

### info

In [9]:
link = "https://store.musinsa.com/app/goods/1199146"
print(link)

https://store.musinsa.com/app/goods/1199146


In [10]:
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [11]:
kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
kw = list(map(lambda s : s.replace("#",''), kw))
kw

['오버핏', '루즈핏', '긴팔티', '스웻셔츠', '맨투맨', '트레이닝세트', '후드티', '세트']

In [12]:
title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
title = title.split(') ')[1]

brand = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()

try:
    o_price = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
except:
    o_price = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()

try:
    s_price = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
except:
    s_price = o_price

kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
kw = list(map(lambda s : s.replace("#",''), kw))

img_link = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()

link = response.url

item_id = link.split('/')[-1]

size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
size_details = {}

for idx in range(len(size_kind)-1):
    sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
    size_details[sd[0]] = sd[1:]
    
sc = response.xpath('//*[@id="size_table"]/thead/tr/th[contains(@class, "item_val")]/text()').extract()
size_category = []
for idx in range(len(sc)):
    if idx % 2 != 0:
        size_category.append(sc[idx].strip())
        
    
# selenium 시작


options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)        
driver.get(link)

try:
    most_age = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/strong[1]')[0].text
except:
    most_age = 0
try:
    most_sex = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/span[1]')[0].text
except:
    most_sex = 0
    
option1_ls = driver.find_elements_by_xpath('//*[@id="option1"]/option[not(@value="")]')
option1 = [option.get_attribute("value") for option in option1_ls]
driver.find_element_by_xpath('//*[@id="option1"]/option[not(@value="")]').click()

option2_ls = driver.find_elements_by_xpath('//*[@id="option2"]/option[not(@value="")]')
option2 = [option.get_attribute("value") for option in option2_ls]


for_size = ['90', '95', 'S', 'M', 'L', 'LARGE', '1', 'M_쭈리']
if any(x in for_size for x in option1):
    sizes = option1
    colors = option2
else:
    sizes = option2
    colors = option1

driver.quit()


# selenium 끝



# 결과 확인

print(
    f"title : {title}" "\n" f'brand : {brand}' "\n"
    f"o_price : {o_price}" "\n" f's_price : {s_price}' "\n"
    f"kw : {kw}" "\n" f'img_link : {img_link}' "\n"
    f"link : {link}" "\n" f'item_id : {item_id}' "\n"
    f"most_age : {most_age}" "\n" f'most_sex : {most_sex}' "\n"
    f"colors : {colors}" "\n" f'sizes : {sizes}' "\n" f'size_details : {size_details}' "\n"
    f"size_category : {size_category}" "\n"
)

title : [패키지] Errday 루즈핏 헤비기모 맨투맨
brand : BRENSON
o_price : 61,200
s_price : 42,900
kw : ['오버핏', '루즈핏', '긴팔티', '스웻셔츠', '맨투맨', '트레이닝세트', '후드티', '세트']
img_link : https://image.msscdn.net/images/goods_img/20191023/1199146/1199146_5_500.jpg
link : https://store.musinsa.com/app/goods/1199146
item_id : 1199146
most_age : 24~28세
most_sex : 남성
colors : ['블랙', '그레이', '멜란지', '네이비', '카키', '더스티그린', '애쉬블루', '크림', '베이지', '코코아', '카멜', '더스티민트', '블루그레이', '애쉬핑크', '브라운']
sizes : ['M', 'L', 'XL', '2XL']
size_details : {'M': ['70', '48', '54', '62.5'], 'L': ['72', '51', '57', '63'], 'XL': ['74', '54', '60', '63.5'], '2XL': ['76', '57', '63', '64']}
size_category : ['총장', '어깨너비', '가슴단면', '소매길이']



In [13]:
# driver.quit()

## 3. items.py

In [14]:
%%writefile musinsa/musinsa/items.py
import scrapy


class MusinsaItem(scrapy.Item):
    title = scrapy.Field()
    brand = scrapy.Field()
    o_price = scrapy.Field()
    s_price = scrapy.Field()
    kw = scrapy.Field()
    img_link = scrapy.Field()
    link = scrapy.Field()
    item_id = scrapy.Field()
    size_category = scrapy.Field()
    size_details = scrapy.Field()

Overwriting musinsa/musinsa/items.py


## 4. spider.py
### 한 페이지 (최대 90개)

In [15]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
from musinsa.items import MusinsaItem
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa"
    allow_domain = ["musinsa.com"]
    start_urls = ["https://search.musinsa.com/category/001003"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 10,
#         'DOWNLOAD_DELAY': 2.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
            'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
        }
    }
    
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = MusinsaItem()
        title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
        item["title"] = title.split(') ')[1]
        item["brand"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
        except:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()
        try:
            item["s_price"] = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
        except:
            item["s_price"] = item["o_price"]
        kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
        item["kw"] = list(map(lambda s : s.replace("#",''), kw))
        item["img_link"] = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()
        item["link"] = response.url
        item["item_id"] = item["link"].split('/')[-1]
        size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
        item["size_details"] = {}
        for idx in range(len(size_kind)-1):
            sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
            item["size_details"][sd[0]] = sd[1:]
            
        sc = response.xpath('//*[@id="size_table"]/thead/tr/th[contains(@class, "item_val")]/text()').extract()
        item["size_category"] = []
        for idx in range(len(sc)):
            if idx % 2 != 0:
                item["size_category"].append(sc[idx].strip())

        yield item

Writing musinsa/musinsa/spiders/spider.py


## 5. run scrapy

In [16]:
%%writefile run.sh
cd musinsa
scrapy crawl Musinsa -o ./datas/top.csv

Writing run.sh


### settings.py 변경

In [17]:
!head -n 20 musinsa/musinsa/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [18]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' musinsa/musinsa/settings.py

In [19]:
!head -n 20 musinsa/musinsa/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = False


#### 주석 해제

In [20]:
# ITEM_PIPELINES = {
#    'musinsa.pipelines.MusinsaPipeline': 300,
# }

## DB 저장

In [21]:
# $ mysql> CREATE DATABASE top
#     CHARACTER SET utf8
#     COLLATE utf8_general_ci

In [29]:
%%writefile musinsa/musinsa/pipelines.py

import mysql.connector
import MySQLdb


class MusinsaPipeline():

    def __init__(self):
        self.create_connection()
        self.create_table()

    def create_connection(self):
        self.conn = mysql.connector.connect(
            host='3.36.165.10',
            user='root',
            passwd='dss',
            database='top',
            charset='utf8',
            use_unicode=True
        )
        self.curr = self.conn.cursor()

    def create_table(self):
        self.curr.execute("""DROP TABLE IF EXISTS item""")
        self.curr.execute("""create table item(
                        title text,
                        brand text,
                        o_price text,
                        s_price text,
                        img_link text,
                        link text,
                        item_id text
                        )""")

    def process_item(self, item, spider):
        self.store_db(item)
        return item

    def store_db(self, item):
        sql = ("""INSERT into item (title, brand, o_price, s_price, img_link, link, item_id)\
VALUES (%s, %s, %s, %s, %s, %s, %s)""")
        query = (
            item["title"],
            item["brand"],
            item["o_price"],
            item["s_price"],
            item["img_link"],
            item["link"],
            item["item_id"]
        )
        self.curr.execute(sql, query)
        self.conn.commit()    

Overwriting musinsa/musinsa/pipelines.py


In [30]:
!source run.sh

2021-03-02 11:26:31 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa)
2021-03-02 11:26:31 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Feb 27 2021, 12:08:28) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-02 11:26:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-02 11:26:31 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'musinsa',
 'CONCURRENT_REQUESTS': 10,
 'NEWSPIDER_MODULE': 'musinsa.spiders',
 'SPIDER_MODULES': ['musinsa.spiders']}
2021-03-02 11:26:31 [scrapy.extensions.telnet] INFO: Telnet Password: fd87a785060cf313
2021-03-02 11:26:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extens

2021-03-02 11:26:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808463> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1808463>
{'brand': 'TOMMY HILFIGER',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210222/1808463/1808463_1_500.jpg',
 'item_id': '1808463',
 'kw': ['T12B0TKT020MT2C9T', 'T12B0TKT020MT2', '코튼레귤러핏솔'],
 'link': 'https://store.musinsa.com/app/goods/1808463',
 'o_price': '138,000',
 's_price': '138,000',
 'size_category': [],
 'size_details': {},
 'title': '코튼 레귤러핏 솔리드 폴로 티셔츠 (T12B0TKT020MT2C9T)'}
2021-03-02 11:26:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808439> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808454> (referer: https://search.musinsa.com/categor

2021-03-02 11:26:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1809389> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1809389>
{'brand': 'SGF67',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210223/1809389/1809389_1_500.jpg',
 'item_id': '1809389',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1809389',
 'o_price': '178,000',
 's_price': '39,000',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'095': ['67.5', '43.5', '42.5', '60.5'],
                  '100': ['69', '45', '44', '61.5'],
                  '105': ['70.5', '46.5', '45.5', '62.5'],
                  '110': ['72', '48', '47', '63.5']},
 'title': '남성 바람개비 프린트 긴팔 카라 티셔츠 와인 (SSX130279)'}
2021-03-02 11:26:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1815612> (referer: https://search.musinsa.com/c

2021-03-02 11:26:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1815607> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1815607>
{'brand': 'YALE',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210225/1815607/1815607_1_500.jpg',
 'item_id': '1815607',
 'kw': ['럭비티', '럭비티셔츠', '럭비셔츠', '오버핏', '핸섬댄', '럭비', '남녀공용'],
 'link': 'https://store.musinsa.com/app/goods/1815607',
 'o_price': '89,000',
 's_price': '75,650',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'L': ['74', '60', '60', '63'],
                  'M': ['72', '58', '58', '62'],
                  'S': ['70', '56', '56', '61'],
                  'XL': ['76', '62', '62', '64']},
 'title': '1701 RUGBY SHIRT BABY BLUE'}
2021-03-02 11:26:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1793083> (referer: https://search.mu

2021-03-02 11:26:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808993> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1808993>
{'brand': 'RENOMA GOLF',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210223/1808993/1808993_1_500.jpg',
 'item_id': '1808993',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1808993',
 'o_price': '179,000',
 's_price': '179,000',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'085': ['57', '33.5', '38.5', '56'],
                  '090': ['58', '35', '41', '57'],
                  '095': ['59', '36.5', '43.5', '58'],
                  '100': ['60', '38', '46', '59']},
 'title': '21SS 여성 사선 지퍼 카라 배색 티셔츠 RWTHK5154-199_G'}
2021-03-02 11:26:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1809020> (referer: https://search.musinsa.com/cat

2021-03-02 11:26:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1809005> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:26:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1809005>
{'brand': 'RENOMA GOLF',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210223/1809005/1809005_1_500.jpg',
 'item_id': '1809005',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1809005',
 'o_price': '139,000',
 's_price': '139,000',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'095': ['67', '41.5', '47.5', '59'],
                  '100': ['68', '43.5', '50', '60'],
                  '105': ['69', '45.5', '52.5', '61'],
                  '110': ['70', '47.5', '55', '62']},
 'title': '21SS 남성 어깨 포인트 카라 티셔츠 RMTYK1151-199_G'}
2021-03-02 11:26:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1809029> (referer: https://search.musinsa.com/c

2021-03-02 11:27:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1387960> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:27:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1387960>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200407/1387960/1387960_1_500.jpg',
 'item_id': '1387960',
 'kw': ['면티셔츠', '반팔티셔츠', '폴로셔츠컬렉션', '하프슬리브', 'MU-T'],
 'link': 'https://store.musinsa.com/app/goods/1387960',
 'o_price': '25,900',
 's_price': '25,900',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'2XL': ['74', '50', '60', '26'],
                  'L': ['71', '47', '55', '24'],
                  'M': ['69.5', '45.5', '52.5', '23'],
                  'S': ['68', '44', '50', '22'],
                  'XL': ['72.5', '48.5', '57.5', '25']},
 'title': '베이식 피케 폴로 셔츠 [블랙]'}
2021-03-02 11:27:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store

2021-03-02 11:27:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1815580> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:27:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1815580>
{'brand': 'Theory',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210225/1815580/1815580_1_500.jpg',
 'item_id': '1815580',
 'kw': ['TM1351C344', 'THEORYMEN'],
 'link': 'https://store.musinsa.com/app/goods/1815580',
 'o_price': '318,000',
 's_price': '318,000',
 'size_category': ['총장', '어깨너비', '가슴단면', '소매길이'],
 'size_details': {'L': ['73.8', '46.3', '52.55', '25'],
                  'M': ['72.5', '45', '50', '23.7'],
                  'S': ['71.2', '43.7', '47.45', '22.4'],
                  'XL': ['75', '48.2', '56.35', '26.6']},
 'title': 'Men Fine Bilen Goris Polo Shirt  Basalt'}
2021-03-02 11:27:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1814010> (referer

2021-03-02 11:27:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1816657> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:27:08 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1816657>
{'brand': 'AND Z',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210226/1816657/1816657_1_500.jpg',
 'item_id': '1816657',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1816657',
 'o_price': '74,000',
 's_price': '74,000',
 'size_category': [],
 'size_details': {},
 'title': '쿨터치 머신워셔블 슬릿넥 카라 니트_BLB2ET1501PGN'}
2021-03-02 11:27:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1816652> (referer: https://search.musinsa.com/category/001003)
2021-03-02 11:27:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1816652>
{'brand': 'AND Z',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210226/1816652/1816652_1_500.jp

2021-03-02 11:27:13 [scrapy.core.engine] INFO: Closing spider (finished)
2021-03-02 11:27:13 [scrapy.extensions.feedexport] INFO: Stored csv feed (89 items) in: ./datas/top.csv
2021-03-02 11:27:13 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 44322,
 'downloader/request_count': 92,
 'downloader/request_method_count/GET': 92,
 'downloader/response_bytes': 3391180,
 'downloader/response_count': 92,
 'downloader/response_status_count/200': 91,
 'downloader/response_status_count/302': 1,
 'elapsed_time_seconds': 42.087525,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 3, 2, 11, 27, 13, 636084),
 'item_scraped_count': 89,
 'log_count/DEBUG': 184,
 'log_count/ERROR': 1,
 'log_count/INFO': 13,
 'memusage/max': 70832128,
 'memusage/startup': 70832128,
 'request_depth_max': 1,
 'response_received_count': 91,
 'scheduler/dequeued': 92,
 'scheduler/dequeued/memory': 92,
 'scheduler/enqueued': 92,
 'scheduler/enqueued/memory': 92,
 'spide

## argument 설정 : sid(카테고리), page(페이지)

In [24]:
%pwd

'/home/ubuntu/notebooks/project/crawling-repo-3'

In [25]:
# %%writefile musinsa/musinsa/spiders/spider.py

# import scrapy
# from musinsa.items import MusinsaItem
# from fake_useragent import UserAgent


# class Spider(scrapy.Spider):
#     name = "Musinsa"
#     allow_domain = ["musinsa.com"]
#     custom_settings = {
#         'CONCURRENT_REQUESTS': 8,
# #         'DOWNLOAD_DELAY': 2.0,
#         'AUTOTHROTTLE_ENABLED': True,
#         "DOWNLOADER_MIDDLEWARES": {
#             "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
#             'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
#             "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
#             'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
#         }
#     }
    
#     def __init__(self, category="001", page="1", **kwargs):
#         self.start_urls = [f"https://search.musinsa.com/category/001{category}?page={page}"]
#         super().__init__(**kwargs)
        
#     def start_requests(self):
#         for url in self.start_urls:
#             yield scrapy.Request(url, callback=self.parse)
    
#     def parse(self, response):
#         links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]/div[2]/p[2]/a/@href').extract()
#         for link in links:
#             yield scrapy.Request(link, callback=self.parse_content)
            
#     def parse_content(self, response):
#         item = MusinsaItem()
#         title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
#         item["title"] = title.split(') ')[1]
#         item["brand"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()
#         try:
#             item["o_price"] = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
#         except:
#             item["o_price"] = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()
#         try:
#             item["s_price"] = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
#         except:
#             item["s_price"] = item["o_price"]
#         kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
#         item["kw"] = list(map(lambda s : s.replace("#",''), kw))
#         item["img_link"] = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()
#         item["link"] = response.url
#         item["item_id"] = item["link"].split('/')[-1]
#         size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
#         item["size_details"] = {}
#         for idx in range(len(size_kind)-1):
#             sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
#             item["size_details"][sd[0]] = sd[1:]

#         yield item

In [26]:
# %%writefile run.sh
# cd ~/python3/notebook/project/crawling-repo-3/musinsa
# scrapy crawl Musinsa -o ./datas/top_a.csv -a page=2

In [27]:
# !/bin/bash run.sh

In [28]:
# sele = pd.read_csv('./musinsa/datas/top_a.csv')